# AtLAST Predictions
Here, you can find another example of using this simulation tool for predictions. But now, in this scenario, we'll adjust the parameters to align with a small instrument that could be mounted on AtLAST. AtLAST will have a broad 2-degree field of view (FOV) and offer a 10" resolution at 150 GHz. This configuration provides a more comprehensive, high spatial dynamic range, ideal for observing phenomena such as the Sunyaev-Zeldovich effect in galaxy clusters.

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy.coordinates import SkyCoord

In [2]:
0.00042211 /7.21287e-07
pixel_size       = 0.0009765625 # degree
pixel_size *3600

3.515625

In [3]:
from maria import Simulation, TOD, utils
from maria.map.mappers import BinMapper

In [4]:
import matplotlib
# plt.style.use('/lustre/home/jvmarrewijk/eszee/Notebooks/thesis.mplstyle')
plt.style.use('/Users/jvanmarr/Documents/Papers/Plot_style/thesis.mplstyle')
matplotlib.rc('image', origin='lower')
cmap = "RdBu_r"

In [5]:
def circle_mask(im, xc, yc, rcirc):
        ny, nx = im.shape
        y,x = np.mgrid[0:nx,0:ny]
        r = np.sqrt((x-xc)*(x-xc) + (y-yc)*(y-yc))
        return ( (r < rcirc))

def r_theta(im, xc, yc):
    # returns the radius rr and the angle phi for point (xc,yc)
    ny, nx = im.shape
    yp, xp = np.mgrid[0:ny,0:nx]
    yp = yp - yc
    xp = xp - xc
    rr = np.sqrt(np.power(yp,2.) + np.power(xp,2.))
    phi = np.arctan2(yp, xp)
    return(rr, phi)    

In [6]:
inputfile = "../maria/maps/big_cluster2.fits"
outfile_map = '/Users/jvanmarr/Documents/Papers/maria/output/AtLAST_cluster.fits'

In [7]:
pointing_center  = (260.0, -10.0) #RA and Dec in degrees

pixel_size       = 0.0009765625 # degree
integration_time = 8.6 * 60.0 #seconds
FREQ             = 92.0 #GHZ

scanning_speed   = 0.5 #deg/s
sample_rate      = float(int(225. * (scanning_speed/0.5) * (FREQ/92.0)**2))  #Hz

field_of_view    = 0.25 # deg
scanning_radius  = field_of_view
ndets_1f         = float(int(3000 * (field_of_view/0.25)**2 * (FREQ/92.0)**2))

ndets            = ndets_1f 
print(f"Should use {int(ndets_1f)} detectors, we use {ndets} dets")
print(f"Sample rate is {int(sample_rate)} Hz")

noisy = False
if noisy:
    atm_model            = "2d"   
    white_noise =  266e-6/np.sqrt(52/37.4) * 0.3 / 1.113
    pink_noise =  0.6*0.3 / 1.113
else:
    atm_model            = None
    white_noise =  0.
    pink_noise =  0.

Should use 3000 detectors, we use 3000.0 dets
Sample rate is 225 Hz


In [8]:
# - Input figure
hdu = fits.open(inputfile)
header = hdu[1].header
wcs_input = WCS(header, naxis=2)

CLUSTER =../DIANOGA/BH2015/D17 / cluster name                                    [astropy.io.fits.card]
OUTTYPE =Comptonization y para / outputtype                                      [astropy.io.fits.card]


### Fine-Tuning Simulation Parameters

To make predictions for AtLAST, several adjustments are required. Firstly, we need to change the pointing center. AtLAST is located near the APEX telescope in the southern hemisphere, so we set the pointing center to a Declination of -10. Additionally, we need to chance the atmospheric conditions. The defeault is set to mid-February at 6 am UT, which is an ideal time for observing with MUSTANG-2 on the GBT but not for AtLAST at Chajnantor. To achieve this, we modify the `start_time` key to August. This change also necessitates adjusting the Right Ascension (RA) of the pointing to ensure that the source remains above the horizon during the observation.

Furthermore, we overwrite the field of view to be 2 degrees and set the scan radius of the daisy scan to 1.3 degrees. We also adjust the detector bandwidth to 52 GHz with a total of 2000 detectors, and set a scan period of 120 seconds. 

Now, it's important to note that we haven't yet developed a mapper that can handle AtLAST's large FOV. Most of the contamination will likely be a common mode in the atmosphere, which needs proper Fourier filtering. This aspect is still a work in progress. Therefore, for the time being, we conduct noiseless observations by setting `atm_model = None`.

In [ ]:
sim = Simulation(
    
    # Mandatory weather settings
    # ---------------------
    instrument = "AtLAST", 
    pointing   = "daisy", 
    site       = "llano_de_chajnantor",  
    
    # Noise settings:
    # ----------------
    atmosphere_model  = atm_model,
    pwv_rms_frac      = 0.07,

    # True sky input
    # ---------------------
    map_file   = None,        
    # map_units  = "Jy/pixel",       
    map_res    = pixel_size,       
    map_center = pointing_center,  
    map_freqs  = [FREQ],
    
    # AtLAST Observational setup
    # ----------------------------
    integration_time = integration_time, 
    sample_rate      = sample_rate,      
    scan_center      = pointing_center,  
    start_time       = "2022-08-01T23:00:00",
    pointing_frame   = "ra_dec", 
    
    field_of_view    = field_of_view,
    bands            = {"f090": {"n_dets": ndets, 
                                 "band_center": FREQ, 
                                 "band_width": 52.0,               
                                 "white_noise": white_noise,
                                 "pink_noise": pink_noise,
                                }
                       },
    
    scan_options     = {"speed":  scanning_speed, 
                        "radius": scanning_radius, 
                        "petals": 2.11
                       },
    verbose=True

)

In [ ]:
tod = sim.run()

### Visualizing the TOD Data

In this section, we present the same array and TOD visualizations as in the MUSTANG-2 case, but this time for AtLAST:

In [ ]:
# visualize scanning patern
# -----------------------
cmap = "RdBu_r"

fig = plt.figure(dpi=256, tight_layout=True)
fig.set_size_inches(12, 5, forward=True)
fig.suptitle("Scanning strategy")

# - Plot
ax = plt.subplot(1, 2, 1)

ax.plot(np.degrees(tod.boresight.az), np.degrees(tod.boresight.el), lw=5e-1)
ax.set_xlabel("az (deg)"), ax.set_ylabel("el (deg)")

ax = plt.subplot(1, 2, 2, projection=wcs_input)
im = ax.imshow(hdu[1].data, cmap=cmap)

ra, dec = ax.coords
ra.set_major_formatter("hh:mm:ss")
dec.set_major_formatter("dd:mm:ss")
ra.set_axislabel(r"RA [J2000]", size=11)
dec.set_axislabel(r"Dec [J2000]", size=11)
ra.set_separator(("h", "m"))

sky = SkyCoord(np.degrees(tod.boresight.ra) * u.deg, np.degrees(tod.boresight.dec) * u.deg)
pixel_sky = wcs_input.world_to_pixel(sky)
ax.plot(pixel_sky[0], pixel_sky[1], lw=5e-1, alpha=0.5)
ax.set_xlabel("ra (deg)"), ax.set_ylabel("dec (deg)")
plt.show()

# visualize powerspectrum
# -----------------------
f, ps = sp.signal.periodogram(tod.data_calibrated, fs=1/(tod.time[1] - tod.time[0]), window="tukey")

fig, axes = plt.subplots(1, 2, figsize=(10, 4), dpi=256, tight_layout=True, gridspec_kw={'width_ratios': [1, 1.6]})
fig.suptitle('Raw time streams')

for i in range(-20,20,2): axes[0].plot(np.logspace(-7, 3,100), np.logspace(-7, 3,100) ** (-8 / 3)/1e11/(10**i), c = 'gray', alpha = 0.2, ls = '--', lw = 0.5)
axes[0].text(1.2e-1,1e-2, r"y = f$^{-(8/3)}$", c = 'gray', alpha = 0.5, rotation = -64)

axes[0].plot(f[1:], ps.mean(axis=0)[1:], label="Real", alpha = 1., c = 'C0', lw = 2, ls = '-')
for key in tod._data.keys():
    f_, ps_ = sp.signal.periodogram(tod._data[key], fs=1/(tod.time[1] - tod.time[0]), window="tukey")
    
    if key == 'map':
        axes[0].plot(f_[1:], ps_.mean(axis=0)[1:], alpha = .7, label= 'data', c = 'C1', lw = 1, ls = '-')
    else:
        axes[0].plot(f_[1:], ps_.mean(axis=0)[1:], alpha = .4, c = 'k', lw = 1, ls = '-')

axes[0].plot(f_[1:], ps_.mean(axis=0)[1:], alpha = .4, c = 'k', lw = 1, ls = '-')

axes[0].set_xlabel(r"$\mathcal{f}$ [Hz]"), 
axes[0].set_ylabel(r"$\mathcal{P}_{\rm Hz}[K_{\rm RJ}^2~{\rm Hz}^{-1}]$")
axes[0].loglog()
axes[0].legend(loc = 1, frameon=False)
axes[0].axis(xmin=1e-3, xmax = 1e3, ymin = 1e-14, ymax = 1e2)

axes[1].set_xlabel('Time [minutes]'), axes[1].set_ylabel("\n"+r'$I_\nu$ [$K_{\rm RJ}$]')
axes[1].plot((tod.time - tod.time[0])/60, tod.data_calibrated[0], label = 'Real', alpha = 1., c = 'C0', lw = 1, ls = '-')
plt.show()

In [ ]:
# visualize scanning patern
# -----------------------
cmap = "RdBu_r"

# visualize powerspectrum
# -----------------------
f, ps = sp.signal.periodogram(tod.data_calibrated, fs=1/(tod.time[1] - tod.time[0]), window="tukey")

fig, axes = plt.subplots(1, 1, figsize=(4, 4.5), dpi=256, tight_layout=True)

for i in range(-20,20,2): axes.plot(np.logspace(-7, 3,100), np.logspace(-7, 3,100) ** (-8 / 3)/1e11/(10**i), c = 'gray', alpha = 0.2, ls = '--', lw = 0.5)
axes.text(2.0e-2,1e-0, r"y = f$^{-(8/3)}$", c = 'gray', alpha = 0.5, rotation = -50)

for key in tod._data.keys():
    
    if key == 'map':
        f_, ps_ = sp.signal.periodogram(tod._data[key]*0.8/ 0.3 * tod.abscal, fs=1/(tod.time[1] - tod.time[0]), window="tukey")
        axes.plot(f_[1:], ps_.mean(axis=0)[1:], alpha = .4, c = f"C3", lw = 1, ls = '-', label = key)
    elif key == 'atmosphere':
        f_, ps_ = sp.signal.periodogram(tod._data[key]*0.8/ 0.3 * tod.abscal, fs=1/(tod.time[1] - tod.time[0]), window="tukey")
        axes.plot(f_[1:], ps_.mean(axis=0)[1:], alpha = .4, c = f"C1", lw = 1, ls = '-', label = key)
    elif key == 'noise':
        f_, ps_ = sp.signal.periodogram(tod._data[key]/ 0.3 * tod.abscal, fs=1/(tod.time[1] - tod.time[0]), window="tukey")
        axes.plot(f_[1:], ps_.mean(axis=0)[1:], alpha = .4, c = f"C2", lw = 1, ls = '-', label = 'white+pink noise')

axes.plot(f[1:], ps.mean(axis=0)[1:], label="Combined", alpha = 0.7, c = 'C0', lw = 1.5, ls = '-')

axes.set_xlabel(r"$\mathcal{f}$ [Hz]"), 
axes.set_ylabel(r"$\mathcal{P}_{\rm Hz}[K_{\rm RJ}^2~{\rm Hz}^{-1}]$")
axes.loglog()
axes.legend(loc = 1, frameon=False)
axes.axis(xmin=1e-3, xmax = 2e2, ymin = 1e-10, ymax = 1e2)
# plt.savefig('/Users/jvanmarr/Documents/Papers/mock_obs/paper_plots/AtLAST_powerspectra.pdf')
plt.show()

## Map-Making

As previously mentioned, we must disable Fourier filtering. Additionally, we have adjusted the height and width of the map to match the realistic AtLAST field of view, which spans several degrees.

In [ ]:
# scan velocity
fov               = (field_of_view*u.degree).to(u.arcsec)
scn_velocity      = scanning_speed * u.degree/u.s
filter_freq       = (scn_velocity/fov).to(u.Hz).value *0.1


In [ ]:
filter_freq 

In [ ]:

mapper = BinMapper(
    center=(tod.coords.center_ra, tod.coords.center_dec),
    frame="ra_dec",
    width=np.radians(1),
    height=np.radians(1),
    res=np.radians(4./3600.),
    degrees=False,
    calibrate = True,
    tod_postprocessing={"remove_modes": {"n": 1}, "highpass": {"f": filter_freq}, "despline":{}},
    map_postprocessing={"gaussian_filter": {"sigma": 2}},
    units="Jy/pixel"
)

mapper.add_tods(tod)
mapper.run()

In [ ]:
fig, axes = plt.subplots(
    1,
    2,
    figsize=(10, 4.5),
    dpi=256,
    tight_layout=True,
    gridspec_kw={"width_ratios": [1, 1.5]},
)
fig.suptitle("Detector setup for one band")

for uband in sim.instrument.bands:
    band_mask = sim.instrument.dets.band == uband

    axes[0].plot(
        60 * np.degrees(tod.boresight.ra - tod.boresight.ra.mean()),
        60 * np.degrees(tod.boresight.dec - tod.boresight.dec.mean()),
        lw=5e-1,
        alpha=0.5,
        label="Scanning Pattern",
    )
    axes[0].scatter(
        60 * np.degrees(sim.instrument.offset_x),
        60 * np.degrees(sim.instrument.offset_y),
        label=f"{uband} mean",
        lw=5e-1,
        c="orange",
    )

axes[0].set_xlabel(r"$\theta_x$ offset (arcminutes)")
axes[0].set_ylabel(r"$\theta_y$ offset (arcminutes)")
# axes[0].legend()

xs, ys = np.meshgrid(
    60 * np.rad2deg((mapper.x_bins[1:] + mapper.x_bins[:-1]) / 2),
    60 * np.rad2deg((mapper.y_bins[1:] + mapper.y_bins[:-1]) / 2),
)

im = axes[1].pcolormesh(
    xs,
    ys,
    mapper.raw_map_cnts[tod.dets.band[0]],
    label="Photon counts in band " + tod.dets.band[0],
)

axes[1].set_xlabel(r"$\theta_x$ (arcmin)"), axes[1].set_ylabel(r"$\theta_y$ (arcmin)")
cbar = plt.colorbar(im, ax=axes[1])
cbar.set_label("Counts")
plt.show()

## Save & Visualize

In [ ]:
mapper.save_maps(outfile_map)

In [ ]:
# lower = SkyCoord(ra = (pointing_center[0]-2.5/60)*u.deg, dec=(pointing_center[1]-2.5/60.0)*u.deg)
# upper = SkyCoord(ra = (pointing_center[0]+2.5/60)*u.deg, dec=(pointing_center[1]+2.5/60.0)*u.deg)

lower = SkyCoord(ra = (pointing_center[0]-0.5/60)*u.deg, dec=(pointing_center[1]-0.5/60.0)*u.deg)
upper = SkyCoord(ra = (pointing_center[0]+0.5/60)*u.deg, dec=(pointing_center[1]+0.5/60.0)*u.deg)

# - Plot Mock observation
outputfile = outfile_map

hdu_out = fits.open(outputfile)
wcs_output = WCS(hdu_out[0].header, naxis=2)

pixel_lower = wcs_output.world_to_pixel(lower)
pixel_upper = wcs_output.world_to_pixel(upper)

sliced_image = hdu_out[0].data[0] #* utils.units.KbrightToJyPix(hdu_out[0].header["CRVAL3"], hdu_out[0].header["CDELT1"], hdu_out[0].header["CDELT2"])
# sliced_image = hdu_out[0].data[0][int(pixel_lower[0]):int(pixel_upper[0]), int(pixel_lower[1]):int(pixel_upper[1])] * utils.units.KbrightToJyPix(hdu_out[0].header["CRVAL3"], hdu_out[0].header["CDELT1"], hdu_out[0].header["CDELT2"])
sliced_wcs = wcs_output[int(pixel_lower[0]):int(pixel_upper[0]), int(pixel_lower[1]):int(pixel_upper[1])] 

# - Plot

fig = plt.figure(dpi=512, tight_layout=False)
fig.set_size_inches(6, 4, forward=True)

ax = plt.subplot(1, 1, 1, projection=wcs_output)
# ax = plt.subplot(1, 1, 1, projection=sliced_wcs)

im = ax.imshow(
    sliced_image * 1e6, cmap=cmap, 
    vmin=-4*np.nanstd(hdu_out[0].data[0]) * 1e6, 
    vmax=4*np.nanstd(hdu_out[0].data[0]) * 1e6,
    rasterized=True
)

cbar = plt.colorbar(im, ax=ax, shrink=1.0)
cbar.set_label(r"S$_{93~\rm GHz}$ [MJy/sr]")

ra, dec = ax.coords
ra.set_major_formatter("hh:mm:ss")
dec.set_major_formatter("dd:mm:ss")
ra.set_axislabel(r"RA [J2000]", size=11)
dec.set_axislabel(r"Dec [J2000]", size=11)
ra.set_separator(("h", "m"))

ax.invert_xaxis()

plt.tight_layout()
# plt.savefig(f'/Users/jvanmarr/Documents/Papers/mock_obs/paper_plots/synthethic_map.pdf', dpi = fig.dpi)
plt.show()

In [ ]:
# tod.to_fits()
# TOD.from_fits()